<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 8
## <center> Vowpal Wabbit в задаче прогнозирования популярности статьи на хабре

В этом задании надо побить бенчмарк в [соревновании](https://www.kaggle.com/c/habr-num-bookmarks) на Kaggle Inclass. Как это делать – ограничений нет (кроме, конечно, ручной разметки), прочитать правила можно [тут](https://www.kaggle.com/c/habr-num-bookmarks/rules). Ниже описаны инструкции, как это сделать с Vowpal Wabbit.

Дедлайн: 31 октября 23:59 UTC +3. Решение надо будет загрузить по [ссылке](https://www.dropbox.com/request/g5WOPrxwvcYwADZCuoY7). В этом соревновании нет задачи победить. Цель – побить бенчмарк и продвинуться в [соревновании](https://mlcourse.arktur.io) по прогнозу популярности статьи на Medium. 

In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error

Посмотрим на одну из строчек в JSON-файле: считаем ее с помощью библиотеки json. Эта строчка соответствует [7-ой статье](https://habrahabr.ru/post/7/) на Хабре.

In [ ]:
!head -2 ../../data/medium/train.json > ../../data/medium/train1.json

In [ ]:
with open('../../data/medium/train1.json') as inp_json:
    first_json = json.load(inp_json)

In [5]:
first_json.keys()

dict_keys(['_id', '_timestamp', 'author', 'content', 'domain', 'flags', 'flow', 'hubs', 'link_tags', 'meta_tags', 'polling', 'post_id', 'published', 'tags', 'title', 'url'])

Видим 16 полей, перечислим некоторые из них:
- _id, url - URL статьи
- published – время публикации статьи
- domain – сайт (например, habrahahbr.ru или geektimes.ru)
- title – название статьи
- content – текст статьи
- hubs - перечисление хабов, к которым относится статья
- tags – теги статьи
- author – автор статьи, его ник и ссылка на профиль

In [6]:
first_json['_id']

'https://habrahabr.ru/post/7/'

In [7]:
first_json['_timestamp']

1493192186.0903192

In [8]:
first_json['url']

'https://habrahabr.ru/post/7/'

In [9]:
first_json['domain']

'habrahabr.ru'

In [10]:
first_json['published']

{'$date': '2006-07-15T01:48:00.000Z'}

In [11]:
first_json['title']

'Самопроизвольное разлогинивание'

In [12]:
first_json['content']

'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.'

In [13]:
first_json['polling']

In [14]:
first_json['post_id']

7

In [15]:
first_json['flags']

[]

In [16]:
first_json['hubs']

[{'id': 'hub/habr',
  'title': 'Хабрахабр',
  'url': 'https://habrahabr.ru/hub/habr/'}]

In [17]:
first_json['flow']

In [18]:
first_json['tags']

['логин', 'login']

In [19]:
first_json['author']

{'name': 'Павел Титов',
 'nickname': '@ptitov',
 'url': 'https://habrahabr.ru/users/ptitov'}

In [20]:
first_json['link_tags']

{'alternate': 'https://habrahabr.ru/rss/post/7/',
 'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
 'canonical': 'https://habrahabr.ru/post/7/',
 'icon': '/images/favicons/favicon-16x16.png',
 'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
 'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'}

In [21]:
first_json['meta_tags']

{'al:android:app_name': 'Habrahabr',
 'al:android:package': 'ru.habrahabr',
 'al:android:url': 'habrahabr://post/7',
 'al:windows_phone:app_id': '460a6bd6-8955-470f-935e-9ea1726a6060',
 'al:windows_phone:app_name': 'Habrahabr',
 'al:windows_phone:url': 'habrahabr://post/7',
 'apple-mobile-web-app-title': 'Хабрахабр',
 'application-name': 'Хабрахабр',
 'description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.\r\n\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'fb:app_id': '444736788986613',
 'keywords': 'логин, login',
 'msapplication-TileColor': '#FFFFFF',
 'msapplication-TileImage': 'mstile-144x144.png',
 'og:description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.  Возможно, это происходит при...',
 'og:image': 'https://habrahabr.ru/i/habralogo.jpg',
 'og:title': 'Самопроизвольное разлогинивание'

Загрузим ответы на обучающей выборке.

In [2]:
train_target = pd.read_csv('../../data/medium/train_target.csv',
                          index_col='url')

In [3]:
train_target.head()

,target
url,
https://habrahabr.ru/post/7/,0.693147
https://geektimes.ru/post/11/,1.098612
https://geektimes.ru/post/112/,0.000000
https://geektimes.ru/post/1127/,0.000000
https://geektimes.ru/post/12664/,0.000000


Сформируйте обучающую выборку для Vowpal Wabbit, выберите признаки title, tags, domain, flow, author, и hubs из JSON-файла.
От самого текста для начала просто возьмем его длину: постройте признак content_len – длина текста в миллионах символов.
Также постройте признаки: час и месяц публикации статьи. Еще, конечно же, возьмите ответы на обучающей выборке из `train_target`. Ниже пример того, как могут выглядеть первые две строки нового файла.

In [23]:
!head -2 ../../data/habr_train.vw

0.6931470000000001 |title Самопроизвольное разлогинивание |tags логин login |domain habrahabr.ru |flow None |author @ptitov |hubs Хабрахабр |num content_len:0.0 month:7 hour:1
1.0986120000000001 |title Stand-along cообщества против сообществ в рамках социальных сетей |tags сообщества интернет-сообщество социальные сети нишевой бренд |domain geektimes.ru |flow None |author @AlexBruce |hubs Чёрная дыра |num content_len:0.0 month:7 hour:14


In [28]:
import math
import datetime

with open('../../data/medium/train.json') as inp_json, \
    open('../../data/medium/habr_train_full.vw', 'w') as out_train:
    
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        target = train_target.loc[data_json['_id']]['target']
        datetime_obj = datetime.datetime.strptime(data_json['published']['$date'], '%Y-%m-%dT%H:%M:%S.%fZ')
        title = str(data_json['title']).replace(":","").replace("|","")
        new_line = str(target) + \
                   ' |title ' + title + \
                   ' |tags ' + ' '.join(data_json['tags']).replace(":","").replace("|","") + \
                   ' |domain ' + str(data_json['domain']) + \
                   ' |flow ' + str(data_json['domain']) + \
                   ' |author ' + str(data_json['author']['nickname']) + \
                   ' |hubs ' + ' '.join(map(lambda x: x['title'], data_json['hubs'])).replace(":","").replace("|","") + \
                   ' |num content_len:' + str(len(data_json['content']) / 1000) + \
                   ' months_trend:' + str(round((datetime_obj.year * 12 + datetime_obj.month)/2000, 3)) + \
                   ' month_sin:' + str(round(math.sin(datetime_obj.month * math.pi / 6), 3)) + \
                   ' month_cos:' + str(round(math.cos(datetime_obj.month * math.pi / 6), 3)) + \
                   ' hour_sin:' + str(round(math.sin(datetime_obj.hour * math.pi / 12), 3)) + \
                   ' hour_cos:' + str(round(math.cos(datetime_obj.hour * math.pi / 12), 3)) + '\n'
        out_train.write(new_line)


A Jupyter Widget

Проделайте все то же с тестовой выборкой, вместо ответов подсовывая что угодно, например, единицы.

In [29]:
with open('../../data/medium/test.json') as inp_json, \
open('../../data/medium/habr_test.vw', 'w') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        target = 1
        title = str(data_json['title']).replace(":","").replace("|","")
        datetime_obj = datetime.datetime.strptime(data_json['published']['$date'], '%Y-%m-%dT%H:%M:%S.%fZ')
        #Плюс для тренда общее число месяцев от рождества Христова/2000
        
        new_line = str(target) + \
                   ' |title ' + title + \
                   ' |tags ' + ' '.join(data_json['tags']).replace(":","").replace("|","") + \
                   ' |domain ' + str(data_json['domain']) + \
                   ' |flow ' + str(data_json['domain']) + \
                   ' |author ' + str(data_json['author']['nickname']) + \
                   ' |hubs ' + ' '.join(map(lambda x: x['title'], data_json['hubs'])).replace(":","").replace("|","") + \
                   ' |num content_len:' + str(len(data_json['content'])/1e6) + \
                   ' months_trend:' + str(round((datetime_obj.year * 12 + datetime_obj.month)/2000, 3)) + \
                   ' month_sin:' + str(round(math.sin(datetime_obj.month * math.pi / 6), 3)) + \
                   ' month_cos:' + str(round(math.cos(datetime_obj.month * math.pi / 6), 3)) + \
                   ' hour_sin:' + str(round(math.sin(datetime_obj.hour * math.pi / 12), 3)) + \
                   ' hour_cos:' + str(round(math.cos(datetime_obj.hour * math.pi / 12), 3)) + '\n'     
        out_vw.write(new_line)


A Jupyter Widget

In [24]:
!head -2 ../../data/habr_test.vw

1 |title День Пи! |tags Пи Pi |domain geektimes.ru |flow None |author @Timursan |hubs Чёрная дыра |num content_len:0.0 month:3 hour:3
1 |title Скрипт для разбиения образов музыкальных CD на треки и конвертации в формат FLAC |tags bash lossless |domain geektimes.ru |flow None |author @da3mon |hubs Чёрная дыра |num content_len:0.01 month:3 hour:0


Выбор того, как валидировать модель, остается за Вами. Проще всего, конечно, сделать отложенную выборку. Бенчмарк, который Вы видите в соревновании (**vw_baseline.csv**) и который надо побить, получен с Vowpal Wabbit, 3 проходами по выборке (не забываем удалять кэш), биграммами и настроенными гиперпараметрами `bits`, `learning_rate` и `power_t`. 

# Train

In [79]:
# separate train to train and holdout for score estimation
train_fraction = 0.7

with open('../../data/medium/habr_train_full.vw') as train_file, \
    open('../../data/medium/habr_train.vw', 'w') as out_train, \
    open('../../data/medium/habr_holdout.vw', 'w') as out_test:
    
    lines = train_file.readlines()
    lines_amount = len(lines)
    split_line = int(train_fraction * lines_amount)
    
    lines_sequence = np.asarray(range(0, lines_amount))
    np.random.seed(17)
    np.random.shuffle(lines_sequence)

    for idx, index in enumerate(lines_sequence):
        if idx < split_line:
            out_train.write(lines[index])
        else:
            out_test.write(lines[index])


`vw -d habr_train.vw -c --passes 3 -k -f habr_model.vw`

- c -- Use a cache. The default is <data>.cache
- k -- delete cache bafore new learning pass
- f -- Final regressor to save (arg is filename)

- --ngram arg    Generate N grams. To target a specific namespace write its name as a prefix to arg 
                         (e.g. --ngram a2 --ngram c3).           
- --loss_function squared
- --bit_precision  # number of bits in the feature table
- --power_t arg (=0.5)         t power value
- --bit_precision 32

### Hyper Parameters Search

In [6]:
# read valid targets 
import re
y = []
with open('../../data/medium/habr_holdout.vw') as pred_file:
    for line in pred_file:
        y.append(float(re.search("^\d\.?(\d+)?", line).group()))
y = np.asarray(y)

In [83]:
from hyperopt import fmin, hp, tpe, Trials

def hyperopt_objective(params):
    !vw -d ../../data/medium/habr_train.vw \
    -l {params['l']} \
    --passes {params['passes']} \
    --bit_precision {params['b']} \
    --ngram 2 \
    --normalized \
    --power_t {params['power_t']} \
    --l2 {params['l2']} \
    --quiet \
    -f ../../data/medium/habr_model_search.vw
    
    !vw -i ../../data/medium/habr_model_search.vw -t \
    -d ../../data/medium/habr_holdout.vw \
    -p ../../data/medium/habr_search_predictions.txt \
    --quiet
    
    with open('../../data/medium/habr_search_predictions.txt') as pred_file:
        prediction = [float(popularity) for popularity in pred_file.readlines()]
    score = mean_absolute_error(y, prediction)
    return score

params_space = {
    'l': hp.uniform('l', 0.2, 0.5), # default = 0.5
    'power_t': hp.uniform('power_t', 0.3, 0.6), # default = 0.5
    'passes': hp.choice('passes', np.arange(1, 3+1, dtype=int)),
    'b': hp.choice('b', np.arange(18, 32+1, dtype=int)),
    'l2': hp.choice('l2', np.arange(0, 100, 10, dtype=int))
}

In [ ]:
%%time
trials = Trials()

best_params = fmin(
    hyperopt_objective,
    space = params_space,
    algo = tpe.suggest,
    max_evals = 100,
    trials = trials
)

In [72]:
best_params

{'b': 7,
 'l': 0.48436911072594774,
 'l2': 4,
 'passes': 1,
 'power_t': 0.32076048348406927}

In [30]:
# Train Vopal Wabbit
!vw -d ../../data/medium/habr_train_shuffled.vw \
-i ../../data/medium/habr_model.vw \
--passes 1 \
--bit_precision 18 \
--ngram 2 \
--power_t 0.32 \
--learning_rate 0.3 \
-f ../../data/medium/habr_model.vw

Generating 2-grams for all namespaces.
final_regressor = ../../data/medium/habr_model.vw
Num weight bits = 18
learning rate = 0.3
initial_t = 0
power_t = 0.32
using no cache
Reading datafile = ../../data/medium/habr_train_shuffled.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.058056 0.058056            1            1.0   3.7377   3.4967       91
0.098920 0.139785            2            2.0   0.6931   0.3193       31
0.188724 0.278529            4            4.0   4.5951   4.4562       55
0.180471 0.172217            8            8.0   1.9459   1.9747       35
0.298672 0.416874           16           16.0   0.0000   0.1788       45
0.492954 0.687236           32           32.0   4.1897   3.3376       45
0.345951 0.198947           64           64.0   1.0986   0.7369       37
0.367160 0.388370          128          128.0   5.0304   4.4240       49
0.297105 0.227050   

# Predict and calculate score

In [31]:
!vw -i ../../data/medium/habr_model.vw -t -d ../../data/medium/habr_holdout.vw \
-p ../../data/medium/test_predictions.txt

Generating 2-grams for all namespaces.
only testing
predictions = ../../data/medium/test_predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../../data/medium/habr_holdout.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.222426 0.222426            1            1.0   4.4188   4.8905       37
1.315150 2.407874            2            2.0   3.3322   1.7805       41
0.865549 0.415949            4            4.0   2.5649   3.2470       33
1.432184 1.998818            8            8.0   3.4340   1.1685       51
1.325224 1.218265           16           16.0   2.3026   1.9629       43
1.060018 0.794811           32           32.0   2.8332   3.0493       53
0.998244 0.936471           64           64.0   4.1431   4.0730       41
1.045088 1.091932          128          128.0   0.0000   0.0000       49
1.008048 0.9

In [32]:
# read valid targets 
y.shape
y

array([ 4.418841,  3.332205,  2.995732, ...,  4.127134,  3.091042,
        3.295837])

In [33]:
# read predictions
predicted = pd.read_table('../../data/medium/test_predictions.txt', header=-1, names='h')
predicted = np.asarray(predicted['h'])
predicted.shape

(36000,)

In [34]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y, predicted)

0.83239817030555552

### History
1. MAE: 0.97,  hour, month, no train shuffle
2. MAE: 0.843,  hour_sin, hour_cos, month_sin, month_cos, train shuffle, 
--cache -k \
--passes 3 \
--bit_precision 26 \
--ngram 2 \
--normalized \
--power_t 0.3 \
--learning_rate 0.1 \
3. MAE: 0.854, +trend_month, Hyperopt {'b': 20.72730122196693,
 'l': 0.36358056461078,
 'passes': 1.7400215119677327,
 'power_t': 0.4893971209575465}
4. MAE: 1.65 :(  Hyperopt + L2
5. MAE: 0.8037

# Prepare submission

In [29]:
# Shuffle train
with open('../../data/medium/habr_train.vw') as train_file, \
    open('../../data/medium/habr_train_shuffled.vw', 'w') as out_train:
    
    lines = train_file.readlines()
    lines_amount = len(lines)
    lines_sequence = np.asarray(range(0, lines_amount))
    np.random.seed(17)
    np.random.shuffle(lines_sequence)

    for idx, index in enumerate(lines_sequence):
        out_train.write(lines[index])


In [18]:
# Train on full set
!vw -d ../../data/medium/habr_train_full_shuffled.vw \
--cache -k \
--passes 3 \
--bit_precision 26 \
--ngram 2 \
--normalized \
--power_t 0.3 \
--learning_rate 0.1 \
-f ../../data/medium/habr_model_full.vw

Generating 2-grams for all namespaces.
final_regressor = ../../data/medium/habr_model_full.vw
Num weight bits = 26
learning rate = 0.1
initial_t = 0
power_t = 0.3
decay_learning_rate = 1
creating cache_file = ../../data/medium/habr_train_full_shuffled.vw.cache
Reading datafile = ../../data/medium/habr_train_full_shuffled.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
14.490676 14.490676            1            1.0   3.8067   0.0000       33
7.573176 0.655676            2            2.0   0.0000   0.8097       23
4.661352 1.749528            4            4.0   0.6931   0.4766       43
6.190978 7.720605            8            8.0   4.9488   0.5731       33
5.931946 5.672914           16           16.0   4.0943   2.8145       57
4.504505 3.077063           32           32.0   5.0626   3.3337       59
3.836956 3.169407           64           64.0   3.9890   2.1413       55

In [35]:
!vw -i ../../data/medium/habr_model.vw -t -d ../../data/medium/habr_test.vw \
-p ../../data/medium/submission_predictions.txt

Generating 2-grams for all namespaces.
only testing
predictions = ../../data/medium/submission_predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../../data/medium/habr_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.007794 0.007794            1            1.0   1.0000   1.0883       23
0.315328 0.622863            2            2.0   1.0000   1.7892       43
0.928896 1.542463            4            4.0   1.0000   2.7453       33
2.449759 3.970623            8            8.0   1.0000   2.8498       31
2.541882 2.634004           16           16.0   1.0000   0.0000       29
2.469460 2.397039           32           32.0   1.0000   2.9229       33
2.179791 1.890121           64           64.0   1.0000   0.6755       37
2.176768 2.173745          128          128.0   1.0000   2.5735       37
1.986002 

In [11]:
sample_sub = pd.read_csv('../../data/medium/sample_submission.csv', 
                         index_col='url')

In [12]:
sample_sub.head()

,target
url,
https://geektimes.ru/post/87455/,11.620054
https://geektimes.ru/post/87452/,4.822528
https://geektimes.ru/post/87459/,0.921104
https://habrahabr.ru/post/87461/,1.632126
https://habrahabr.ru/post/5754/,1.952122


In [13]:
your_submission = sample_sub.copy()
your_submission.head()

,target
url,
https://geektimes.ru/post/87455/,11.620054
https://geektimes.ru/post/87452/,4.822528
https://geektimes.ru/post/87459/,0.921104
https://habrahabr.ru/post/87461/,1.632126
https://habrahabr.ru/post/5754/,1.952122


In [36]:
predicted = pd.read_table('../../data/medium/submission_predictions.txt', header=-1, names='h')
predicted = np.asarray(predicted['h'])
print(predicted.shape[0], your_submission.index.shape[0]) 

52913 52913


In [37]:
your_submission['target'] = predicted
your_submission.to_csv('../../data/medium/submission.csv')

Для получения баллов в #mlcourse_open команда (из 1 человека) должна называться в точном соответствии с тем, как оно записано в рейтинге.